In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
%matplotlib widget
import matplotlib.pyplot as plt
import seaborn as sns

In [3]:
import numpy as np
import pandas as pd
import xarray as xr

In [4]:
from ecephys_analyses.data import paths
from ecephys.scoring import load_datetime_hypnogram, filter_states
from ecephys.utils import load_df_h5, replace_outliers
from ecephys.signal.sharp_wave_ripples import get_events_in_interval
import ecephys.plot as eplt
from neurodsp.plts.time_series import plot_time_series

In [5]:
from ripple_detection.core import gaussian_smooth

def my_gaussian_smooth(da, smoothing_sigma=10):
    estimated_fs = 1 / np.diff(da.time.values).mean()
    return gaussian_smooth(da, smoothing_sigma, estimated_fs)

In [11]:
def load_condition_data(subject, condition):
    hypnogram_paths = paths.get_sglx_style_datapaths(subject=subject, condition=condition, ext="hypnogram.tsv")
    hypnograms = [load_datetime_hypnogram(path) for path in hypnogram_paths]
    hypnogram = pd.concat(hypnograms).reset_index(drop=True)
    condition_start_dt = hypnogram['start_time'].iloc[0]
    
    spw_paths = paths.get_sglx_style_datapaths(subject=subject, condition=condition, ext="spws.h5")
    spws = [load_df_h5(path) for path in spw_paths]
    
    for _spws, _hypnogram in zip(spws, hypnograms):
        file_start_dt = _hypnogram['start_time'].min()
        _spws['start_time'] = file_start_dt + pd.to_timedelta(_spws['start_time'], 's')
        _spws['end_time'] = file_start_dt + pd.to_timedelta(_spws['end_time'], 's')
        _spws['midpoint'] = file_start_dt + pd.to_timedelta(_spws['midpoint'], 's')
        #spws['duration'] = pd.to_timedelta(spws['duration'], 's')
        _spws['time_from_condition_start'] = _spws['start_time'] - condition_start_dt
    
    combined_spws = pd.concat(spws).reset_index(drop=True)
    combined_spws.index += 1
    combined_spws.index = combined_spws.index.rename('spw_number')
    spws = combined_spws
    
    bandpower_paths = paths.get_sglx_style_datapaths(subject=subject, condition=condition, ext="pow.nc")
    bandpowers = [xr.open_dataset(path) for path in bandpower_paths]
    for bp in bandpowers:
        bp.mpta_delta.values = my_gaussian_smooth(bp.mpta_delta, smoothing_sigma=10)
        bp.sr_delta.values = my_gaussian_smooth(bp.sr_delta, smoothing_sigma=10)
        bp['time'] = pd.to_datetime(bp.mpta_delta.file_start) + pd.to_timedelta(bp.time.values, 's')
    bandpower = xr.concat(bandpowers, dim="time")
    
    return hypnogram, spws, bandpower

In [12]:
subject = "Eugene"
bsl_condition = "sleep-deprivation-12h-circadian-match"
exp_condition = "sleep-deprivation-12h"

In [27]:
MPTA_SWA_LIM = (0, 100000)
SR_SWA_LIM = (0, 120000)
DENSITY_LIM = (0, 20)
DURATION_LIM = (0, 0.3)
AMPLITUDE_LIM = (-0.08, -0.01)
INTEGRAL_LIM = (-6.0, 0)

In [28]:
(bsl_hypnogram, bsl_spws, bsl_power) = load_condition_data(subject, bsl_condition)
(exp_hypnogram, exp_spws, exp_power) = load_condition_data(subject, exp_condition)

In [29]:
def plot_condition(hypnogram, spws, power):

    fig, (mpta_swa_ax, sr_swa_ax, density_ax, duration_ax, amplitude_ax, integral_ax) = plt.subplots(6, 1, figsize=(22,12))

    xmargin=0

    sns.lineplot(x=power.time.values, y=power.mpta_delta.values, color='black', ax=mpta_swa_ax)
    mpta_swa_ax.set(xlabel=None, ylabel='CTX SWA (uV^2/Hz)', ylim=MPTA_SWA_LIM, xmargin=xmargin, xticks=[])

    sns.lineplot(x=power.time.values, y=power.sr_delta.values, color='black', ax=sr_swa_ax)
    sr_swa_ax.set(xlabel=None, ylabel='HPC SWA (uV^2/Hz)', ylim=SR_SWA_LIM, xmargin=xmargin, xticks=[])

    binwidth = 10
    nbins = np.int((spws.start_time.max() - spws.start_time.min()).total_seconds() / binwidth)
    sns.histplot(data=spws, x='start_time', bins=nbins, color='black', ax=density_ax)
    density_ax.set(xlabel=None, ylabel='Frequency (Hz)', ylim=DENSITY_LIM, xmargin=xmargin, xticks=[])

    sns.scatterplot(data=spws, x="start_time", y="duration", s=2, ax=duration_ax)
    duration_ax.set(xlabel=None, ylabel="Duration (s)", ylim=DURATION_LIM, xmargin=xmargin, xticks=[])

    sns.scatterplot(data=spws, x='start_time', y='sink_amplitude', s=2, ax=amplitude_ax)
    amplitude_ax.set(xlabel=None, ylabel="Sink amplitude (mA/mm)", ylim=AMPLITUDE_LIM, xmargin=xmargin, xticks=[])

    sns.scatterplot(data=spws, x='start_time', y='sink_integral', s=2, ax=integral_ax)
    integral_ax.set(xlabel="Event time (month-date hour)", ylabel="Sink integral (mA * ms)", ylim=INTEGRAL_LIM, xmargin=xmargin)

    for ax in (mpta_swa_ax, sr_swa_ax, density_ax, duration_ax, amplitude_ax, integral_ax):
        eplt.plot_hypnogram_overlay(hypnogram, ax=ax)

    plt.tight_layout()

In [30]:
plot_condition(bsl_hypnogram, bsl_spws, bsl_power)

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [31]:
plot_condition(exp_hypnogram, exp_spws, exp_power)

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [32]:
def plot_both_conditions(bsl_hypnogram, bsl_spws, bsl_power, exp_hypnogram, exp_spws, exp_power):

    fig, (bsl_swa_ax, bsl_density_ax, bsl_amplitude_ax, exp_swa_ax, exp_density_ax, exp_amplitude_ax) = plt.subplots(6, 1, figsize=(22,12))

    xmargin=0

    # Baseline
    sns.lineplot(x=bsl_power.time.values, y=bsl_power.mpta_delta.values, color='black', ax=bsl_swa_ax)
    bsl_swa_ax.set(xlabel=None, ylabel='BSL CTX SWA (uV^2/Hz)', ylim=MPTA_SWA_LIM, xmargin=xmargin, xticks=[])

    binwidth = 10
    nbins = np.int((bsl_spws.start_time.max() - bsl_spws.start_time.min()).total_seconds() / binwidth)
    sns.histplot(data=bsl_spws, x='start_time', bins=nbins, color='black', ax=bsl_density_ax)
    bsl_density_ax.set(xlabel=None, ylabel='BSL Frequency (Hz)', ylim=DENSITY_LIM, xmargin=xmargin, xticks=[])
    
    sns.scatterplot(data=bsl_spws, x='start_time', y='sink_amplitude', s=2, ax=bsl_amplitude_ax)
    bsl_amplitude_ax.set(xlabel=None, ylabel="BSL Sink amp. (mA/mm)", ylim=AMPLITUDE_LIM, xmargin=xmargin, xticks=[])
    
    for ax in (bsl_swa_ax, bsl_density_ax, bsl_amplitude_ax):
        eplt.plot_hypnogram_overlay(bsl_hypnogram, ax=ax)

    # Experiment
    sns.lineplot(x=exp_power.time.values, y=exp_power.mpta_delta.values, color='black', ax=exp_swa_ax)
    exp_swa_ax.set(xlabel=None, ylabel='EXP CTX SWA (uV^2/Hz)', ylim=MPTA_SWA_LIM, xmargin=xmargin, xticks=[])

    binwidth = 10
    nbins = np.int((exp_spws.start_time.max() - exp_spws.start_time.min()).total_seconds() / binwidth)
    sns.histplot(data=exp_spws, x='start_time', bins=nbins, color='black', ax=exp_density_ax)
    exp_density_ax.set(xlabel=None, ylabel='EXP Frequency (Hz)', ylim=DENSITY_LIM, xmargin=xmargin, xticks=[])
    
    sns.scatterplot(data=exp_spws, x='start_time', y='sink_amplitude', s=2, ax=exp_amplitude_ax)
    exp_amplitude_ax.set(xlabel=None, ylabel="EXP Sink amp. (mA/mm)", ylim=AMPLITUDE_LIM, xmargin=xmargin, xticks=[])
        
    for ax in (exp_swa_ax, exp_density_ax, exp_amplitude_ax):
        eplt.plot_hypnogram_overlay(exp_hypnogram, ax=ax)

    plt.tight_layout()

In [33]:
plot_both_conditions(bsl_hypnogram, bsl_spws, bsl_power, exp_hypnogram, exp_spws, exp_power)

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …